In [1]:
import requests
from dataclasses import dataclass
from typing import Literal
from datetime import datetime
from decimal import Decimal

import pytz
from decouple import config
from urllib.parse import urlencode

In [15]:
ALPHAVANTAGE_API_KEY = config('ALPHAVANTAGE_API_KEY', default=None, cast=str)

def transform_alpha_vantage_result(timestamp_str, result):
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    eastern_timezone = pytz.timezone('US/Eastern')
    utc_timezone = pytz.utc

    timestamp = eastern_timezone.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc_timezone)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),        
        "number_of_transactions": None,
        'volume': int(result['5. volume']),
        "volume_weighted_average": None,
        "time": timestamp
    }


@dataclass
class AlphaVantageAPIClient:
    ticker: str = 'AAPL'
    function: Literal["TIME_SERIES_INTRADAY", "TIME_SERIES_DAILY", "TIME_SERIES_WEEKLY", "TIME_SERIES_MONTHLY"] = "TIME_SERIES_INTRADAY"
    interval: Literal["1min", "5min", "15min", "30min", "60min"] = "1min"
    adjusted: bool = True
    
    multiplier: int = 5
    timespan: str = "day"
    from_date: str = "2023-01-09"
    to_date: str = "2023-01-10"
    api_key: str = ""
    adjusted: bool = True
    

    def get_api_key(self):
        return self.api_key or ALPHAVANTAGE_API_KEY
        
    def get_headers(self):
        api_key = self.get_api_key()
        return {
            "Authorization": f"Bearer {api_key}"
        }
    
    def get_params(self):
        return {
            "apikey": self.get_api_key(),
            "symbol": self.ticker,
            "interval": self.interval,
            "function": self.function,
            # "adjusted": self.adjusted,
        }
    
    def generate_url(self, pass_auth: bool = False):
        url = f"https://www.alphavantage.co/query"
        params = self.get_params()
        encoded_params = urlencode(params)        
        url = f"{url}?{encoded_params}"
        if pass_auth:
            url = f"{url}&api_Key={self.get_api_key()}"
        return url

    def fetch_data(self):
        url = self.generate_url()
        headers = self.get_headers()
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        dataset_keys = [x for x in data.keys() if not x.startswith('Meta Data')]
        # print(dataset_keys)
        results = data[dataset_keys[0]]
        return [transform_alpha_vantage_result(timestamp_str, result) for timestamp_str, result in results.items() if timestamp_str != 'Meta Data']


In [16]:
AlphaVantageAPIClient().get_stock_data()


[{'open_price': Decimal('258.1300'),
  'close_price': Decimal('258.1600'),
  'high_price': Decimal('258.1600'),
  'low_price': Decimal('258.1000'),
  'number_of_transactions': None,
  'volume': 335,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 12, 24, 21, 59, tzinfo=<UTC>)},
 {'open_price': Decimal('258.1100'),
  'close_price': Decimal('258.1101'),
  'high_price': Decimal('258.1600'),
  'low_price': Decimal('258.0900'),
  'number_of_transactions': None,
  'volume': 648,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 12, 24, 21, 58, tzinfo=<UTC>)},
 {'open_price': Decimal('258.1200'),
  'close_price': Decimal('258.1300'),
  'high_price': Decimal('258.1600'),
  'low_price': Decimal('258.1050'),
  'number_of_transactions': None,
  'volume': 123,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 12, 24, 21, 57, tzinfo=<UTC>)},
 {'open_price': Decimal('258.1400'),
  'close_price': Decimal('258.1050'),
  'high_price': Decimal('